(BasicLoading)=
# Basic file loading operations

The core of all the operations is the `FileManager` class defined in the `read_input` submodule. The `FileManager` class can load both `.DTA` and `.mpt` files generated respectively by the GAMRY and BIOLOGIC potentiostats providing a simple interface to automatically parse all the loaded data. To show how this can be done let us consider the case of a `.mpt` file that, for the purposes of this example, is provided in the documentation folder. Let us look at the content of such file:

In [1]:
with open("./example_BIOLOGIC.mpt", 'r') as file:
    for line in file:
        print(line.rstrip('\n'))

...
Acquisition started on : 25/12/2022 13:00:00
...
Number of loops : 2
Loop 0 from point number 0 to 5
Loop 1 from point number 6 to 11

mode	ox/red	error	control changes	time/s	control/V/mA	Ewe/V	I/mA	dq/mA.h	(Q-Qo)/mA.h	Q charge/discharge/mA.h	Ece/V	P/W	Q discharge/mA.h	Q charge/mA.h	Capacity/mA.h	control/V	control/mA	Ewe-Ece/V	
1	1	0	1	1,000000000000000E+002	8,0000000E+002	1,0000000E+000	8,0000000E+002	4,000000000000000E+000	1,100000000000000E+003	1,100000000000000E+003	3,0000000E-006	1,0000000E+000	0,000000000000000E+000	1,000000000000000E+003	1,000000000000000E+003	0,0000000E+000	8,0000000E+002	1,0000000E+000
1	1	0	1	1,010000000000000E+002	8,0000000E+002	1,1000000E+000	8,0000000E+002	4,000000000000000E+000	1,100000000000000E+003	1,100000000000000E+003	3,0000000E-006	1,0000000E+000	0,000000000000000E+000	1,000000000000000E+003	1,000000000000000E+003	0,0000000E+000	8,0000000E+002	1,0000000E+000
1	1	0	1	1,020000000000000E+002	8,0000000E+002	1,2000000E+000	8,0000000E+002	4,000000000

As can be seen the provided file describes two full charge/discharge cycles each of wich composed by two halfcycle composed by 3 time-steps each.

To load this `.mpt` file using the `FileManager` class, the following procedure can be used:

In [2]:
# Loading the FileManager class from the cellcycling.read_input module
from echemsuite.cellcycling.read_input import FileManager

# Creating an instance of the FileManager class
manager = FileManager(verbose=False)

# Loading all the .mpt file in the current folder
manager.fetch_from_folder("./", extension=".mpt")   

After the call to the `fetch_from_folder` function, all the data has been loaded in the class and parsed in single independent half-cycles.Considering that we have loaded a single `.mpt` file, the half-cycle ordering is not ambiguos and the parsed data can immediately be converted in the corresponding `Cycle` and `CellCycling` objects by using the built in `get_cycles` and `get_cellcycling` class methods according to:

In [3]:
# Obtain the cellcycling object from the manager
cellcycling = manager.get_cellcycling()
print(cellcycling)

# Obtain the list of cycle objects from the manager
cycle_list = manager.get_cycles()
print(cycle_list[0])

If the cell-cycling data is instead loaded from a set of `.DTA` files, either complete or partial, the ordering of the half-cycles is not strictly encoded by the data available to the `FileManager` and, as such, user intervention may be needed. The `FileManager` will try to automatically generate a suggested ordering of the half-cycle files based on their timestamp and type. The suggested ordering can be obtained by calling the memeber function `suggest_ordering` that will return a list of lists containing the name of the loaded files. Each list-type entry represents an half-cycle level and list with more than one element will represent patial half-cycle files to be merged. As an example consider the followiing list:

```
[["Charge_1.DTA"], ["Charge_2.DTA", "Charge_2b.DTA"], ["Charge_3.DTA"]]
```

this list represents an ordering that will generate 3 half-cycles in which the middle one will be generated from the merging of `Charge_2.DTA` and `Charge_2b.DTA`. The user can either accept the automatically generated ordering or can manually specify its own. The selected ordering can then be applyed by passing it explicitly to the `get_cycles` and `get_cellcycling` functions as the keyworded argument `custom_order`. 

## Loading data from a BytesIO bytestream

The `read_input` sub-module has been developed to be able to also operate in a web-application environment minimizing the operations done on local copies of the loaded files. For this reason, all the loaded files are immediately converted in binary bytestreams and saved in a `bytestrem` dictionary saved in memory. This behavior opens, as a direct consequence, an alternative way to define a parametrized `FileManager` object by direct specification, thanks to the provided setter, of the `bytestream` buffer.

To show how this can be done let us, first of all, define a `BytesIO` version of the `.mpt` file considered before. This can easily be done with few lines of code:

In [4]:
# Load the BytesIO stream object from the io module
from io import BytesIO

# Open the file and load it as a bytes stream
my_bytestream = None
with open("./example_BIOLOGIC.mpt", 'rb') as binary_file:
    my_bytestream = BytesIO(binary_file.read())

The loaded stream can now be used to directly initialize the `FileManager` class according to:

In [5]:
from echemsuite.cellcycling.read_input import Instrument

# Create a FileManager instance
manager = FileManager()

# Set the bytestream dictionary using the loaded data
manager.bytestreams = {"example_BIOLOGIC.mpt" : my_bytestream}
manager._instrument = Instrument.BIOLOGIC

# Call the parse function
manager.parse()

# Just to show that the parsing worked print the loaded halfcycles
for i, (name, halfcycle) in enumerate(manager.halfcycles.items()):
    print(f"{i}) {name}\t{halfcycle}")

0) charge_0_example_BIOLOGIC.mpt	<echemsuite.cellcycling.cycles.HalfCycle object at 0x7f88e6778eb0>
1) discharge_0_example_BIOLOGIC.mpt	<echemsuite.cellcycling.cycles.HalfCycle object at 0x7f88e6779750>
2) charge_1_example_BIOLOGIC.mpt	<echemsuite.cellcycling.cycles.HalfCycle object at 0x7f88e6779c30>
3) discharge_1_example_BIOLOGIC.mpt	<echemsuite.cellcycling.cycles.HalfCycle object at 0x7f88e677a4a0>


Please observe how the `FileManager` class has no access to the file extension (the label provided here as a dictionary key was used only for clarity) and, as a direct consequence, does not know the instrument type. This requires the user to manually set the `_instrument` property. Furthermore, unlike the previous example in which the call to the `parse` function was avoided thanks to the `autoparse` mode of the `fetch_from_folder` function, this time the call to the `parse` function must be made explicitly. 